In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,103.03,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,-0.0,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00
2,-0.00,138.85,-0.00,0.01,-0.00,0.00,-0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,-0.00,0.00,0.01,0.00,0.00,0.00,0.00
3,-0.00,-0.00,59.35,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,...,-0.00,-0.0,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00
4,-0.00,0.01,-0.00,146.89,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,0.0,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00
5,0.00,-0.00,-0.00,0.00,61.76,0.00,-0.00,0.00,-0.00,0.00,...,0.00,0.0,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
6,0.00,0.00,0.00,0.00,0.00,53.91,-0.00,-0.00,-0.00,-0.00,...,-0.00,0.0,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00
7,0.00,-0.00,0.00,0.00,-0.00,-0.00,73.28,0.00,-0.00,0.00,...,0.00,-0.0,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00
8,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,2.38,-0.00,0.00,...,-0.00,-0.0,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00
9,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,21.85,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00
10,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,53.76,...,0.00,-0.0,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,-0.00002,-0.00004,-0.00003,0.00005,0.00004,0.00001,0.00000,0.00002,0.00002,...,-0.00001,-0.00002,0.00002,0.00000,0.00003,-0.00004,0.00002,0.00003,-0.00001,0.00000
2,-0.00002,1.00000,-0.00001,0.00004,-0.00002,0.00002,-0.00003,0.00000,0.00004,0.00002,...,0.00004,0.00003,0.00001,-0.00001,0.00004,0.00006,0.00006,0.00005,0.00002,0.00001
3,-0.00004,-0.00001,1.00000,-0.00002,-0.00000,0.00002,0.00002,-0.00001,-0.00001,0.00001,...,-0.00002,-0.00001,0.00005,0.00003,-0.00001,0.00004,-0.00003,-0.00001,-0.00000,-0.00003
4,-0.00003,0.00004,-0.00002,1.00000,0.00001,0.00004,0.00002,0.00001,0.00003,0.00001,...,-0.00002,0.00001,-0.00000,-0.00004,0.00001,-0.00002,0.00002,-0.00001,0.00002,-0.00005
5,0.00005,-0.00002,-0.00000,0.00001,1.00000,0.00001,-0.00002,0.00000,-0.00000,0.00005,...,0.00003,0.00000,-0.00000,-0.00001,0.00003,0.00003,-0.00000,0.00002,-0.00002,-0.00004
6,0.00004,0.00002,0.00002,0.00004,0.00001,1.00000,-0.00003,-0.00002,-0.00010,-0.00002,...,-0.00003,0.00004,0.00001,0.00001,-0.00004,0.00005,-0.00001,-0.00002,0.00003,-0.00005
7,0.00001,-0.00003,0.00002,0.00002,-0.00002,-0.00003,1.00000,0.00001,-0.00001,0.00000,...,0.00002,-0.00001,0.00003,-0.00001,-0.00005,0.00003,0.00002,-0.00002,0.00001,-0.00007
8,0.00000,0.00000,-0.00001,0.00001,0.00000,-0.00002,0.00001,1.00000,-0.00001,0.00007,...,-0.00006,-0.00005,-0.00002,0.00002,0.00004,-0.00001,0.00001,-0.00001,-0.00001,0.00003
9,0.00002,0.00004,-0.00001,0.00003,-0.00000,-0.00010,-0.00001,-0.00001,1.00000,0.00003,...,0.00000,0.00001,0.00006,0.00004,0.00000,0.00005,-0.00005,-0.00001,-0.00004,-0.00001
10,0.00002,0.00002,0.00001,0.00001,0.00005,-0.00002,0.00000,0.00007,0.00003,1.00000,...,0.00001,-0.00001,-0.00004,0.00000,0.00003,-0.00004,-0.00004,-0.00002,-0.00003,0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.15175477294957582

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.16100876e+03 2.00070892e+03 9.50396653e+02 6.86111225e+02
 3.61569677e+02 3.22852991e+02 2.80652823e+02 2.73799257e+02
 2.62821626e+02 2.49456864e+02 2.29593197e+02 1.78313648e+02
 1.70257547e+02 1.62106650e+02 1.56488934e+02 1.44160021e+02
 1.36619512e+02 1.10086389e+02 1.02720513e+02 1.00927663e+02
 9.71061537e+01 9.15816921e+01 8.91951617e+01 8.46456902e+01
 7.89623023e+01 7.67377807e+01 7.47035110e+01 7.43345857e+01
 7.36875909e+01 6.86047932e+01 6.33740236e+01 5.99933530e+01
 5.79725923e+01 5.45619003e+01 5.01781199e+01 4.91230712e+01
 4.74032225e+01 4.53601784e+01 4.45613141e+01 4.15589923e+01
 3.80707782e+01 3.47323646e+01 3.30407951e+01 3.20922583e+01
 3.06269468e+01 2.80220612e+01 2.61016034e+01 2.32172272e+01
 2.10767802e+01 1.99749008e+01 1.70157474e+01 1.29520528e+01
 1.00343990e+01 8.03566178e+00 7.40372552e+00 6.80261025e+00
 6.10282782e+00 1.41558717e+00 2.78997334e-01 1.10459490e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,-0.003827,-0.005646,-0.002094,-0.006068,-0.002158,-0.001887,-0.002617,-0.000082,-0.000756,-0.001878,...,-0.000675,-0.001533,-0.002934,-0.001727,-0.001478,-0.003617,-0.000200,-0.001343,-0.002122,-0.000639
1,-0.006153,-0.009055,-0.003317,-0.009823,-0.003482,-0.003018,-0.004197,-0.000130,-0.001197,-0.003006,...,-0.001084,-0.002472,-0.004668,-0.002743,-0.002410,-0.005832,-0.000309,-0.002160,-0.003344,-0.001019
2,-0.008244,-0.013622,-0.004069,-0.015328,-0.004285,-0.003714,-0.005246,-0.000153,-0.001433,-0.003673,...,-0.001288,-0.002981,-0.005926,-0.003300,-0.002899,-0.007720,-0.000384,-0.002584,-0.004109,-0.001202
3,-0.010969,-0.020816,-0.005048,-0.024589,-0.005331,-0.004533,-0.006621,-0.000186,-0.001743,-0.004500,...,-0.001577,-0.003657,-0.007526,-0.004045,-0.003530,-0.010118,-0.000446,-0.003162,-0.005123,-0.001467
4,-0.015783,-0.109732,-0.005461,0.989201,-0.005746,-0.004746,-0.007463,-0.000180,-0.001700,-0.004749,...,-0.001558,-0.003745,-0.008962,-0.004232,-0.003620,-0.013774,-0.000433,-0.003209,-0.005458,-0.001470
5,-0.019989,0.983194,-0.006235,0.093988,-0.006629,-0.005410,-0.008773,-0.000203,-0.001900,-0.005400,...,-0.001712,-0.004212,-0.010538,-0.004774,-0.004058,-0.016824,-0.000475,-0.003570,-0.006243,-0.001622
6,-0.011759,0.034393,-0.003082,0.020087,-0.003299,-0.002746,-0.004398,-0.000090,-0.000905,-0.002645,...,-0.000823,-0.002048,-0.005475,-0.002335,-0.002015,-0.009668,-0.000235,-0.001739,-0.003107,-0.000765
7,-0.023657,0.056263,-0.005916,0.034391,-0.006269,-0.005135,-0.008579,-0.000189,-0.001743,-0.005115,...,-0.001584,-0.003940,-0.010668,-0.004482,-0.003802,-0.019092,-0.000448,-0.003366,-0.006040,-0.001506
8,-0.032310,0.055998,-0.007587,0.036401,-0.008092,-0.006548,-0.011050,-0.000243,-0.002223,-0.006495,...,-0.002032,-0.005011,-0.013951,-0.005683,-0.004880,-0.025465,-0.000567,-0.004266,-0.007584,-0.001868
9,-0.052816,0.063557,-0.011156,0.045000,-0.011905,-0.009588,-0.016455,-0.000339,-0.003219,-0.009577,...,-0.002930,-0.007326,-0.020798,-0.008318,-0.007114,-0.040785,-0.000807,-0.006173,-0.011190,-0.002721


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0038272574931661,
 1.0090545999012086,
 1.0040689696929803,
 1.0245894916685163,
 1.0057456829053633,
 1.0054104815383842,
 1.00439830220603,
 1.0001891747227982,
 1.002223335295963,
 1.009576634737615,
 1.011073331709187,
 1.0044635475680141,
 1.0088266722334025,
 1.0115979337611705,
 1.024058926688232,
 1.006159176714372,
 1.0189905839155349,
 1.0043055337186926,
 0.9982131864237793,
 0.9955221605609295,
 0.9970435043382333,
 1.0058323479266786,
 0.9856667722774024,
 0.9070798591978632,
 1.003046767609032,
 1.0087587397139595,
 1.0005384407757725,
 1.0009982329939817,
 1.0090914731273275,
 1.2275532192660632,
 0.9460553156922769,
 0.9964277589290021,
 0.9933368711981162,
 0.9632245481435361,
 0.9983059755102268,
 0.9996894833508458,
 0.9868041903598586,
 0.9944341928586029,
 1.0060039431649053,
 0.9637068429370733,
 0.9846982257737835,
 1.1140310374557119,
 0.9967197899622833,
 1.0101292991256583,
 0.9625636655379969,
 1.0644964093619191,
 0.942442342579157,
 1.0131134034066716,
 